In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 
import tensorflow as tf

import textstat
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import torch
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
#from transformers import AdamW
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import load_model
import json


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import AdamW
#from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [ ]:
df_tr_01 = pd.read_csv('training_data.csv')
print(len(df_tr_01))
print(df_tr_01.head())

In [ ]:
df_unte_01 = pd.read_csv('unlabelled_test_data.csv')
print(len(df_unte_01))
print(df_unte_01.head())

In [ ]:
print(df_tr_01.shape)
print(df_tr_01.info())
print('----------')
df_tr_02 = df_tr_01.copy()
for i in range(0, len(df_tr_02.difficulty)):
    if df_tr_02.iloc[i,2] == "A1":
        df_tr_02.iloc[i,2] = int(1)
    elif df_tr_02.iloc[i,2] == "A2":
        df_tr_02.iloc[i,2] =int(2)
    elif df_tr_02.iloc[i,2] == "B1":
        df_tr_02.iloc[i,2] = int(3)
    elif df_tr_02.iloc[i,2] == "B2":
        df_tr_02.iloc[i,2] = int(4)
    elif df_tr_02.iloc[i,2] == "C1":
        df_tr_02.iloc[i,2] = int(5)
    elif df_tr_02.iloc[i,2] == "C2":
        df_tr_02.iloc[i,2] = int(6)
df_tr_02.difficulty.unique()

df_tr_05_02 = df_tr_01.copy()
for i in range(0, len(df_tr_02.difficulty)):
    if df_tr_05_02.iloc[i,2] == "A1":
        df_tr_05_02.iloc[i,2] = int(0)
    elif df_tr_05_02.iloc[i,2] == "A2":
        df_tr_05_02.iloc[i,2] =int(1)
    elif df_tr_05_02.iloc[i,2] == "B1":
        df_tr_05_02.iloc[i,2] = int(2)
    elif df_tr_05_02.iloc[i,2] == "B2":
        df_tr_05_02.iloc[i,2] = int(3)
    elif df_tr_05_02.iloc[i,2] == "C1":
        df_tr_05_02.iloc[i,2] = int(4)
    elif df_tr_05_02.iloc[i,2] == "C2":
        df_tr_05_02.iloc[i,2] = int(5)
df_tr_05_02.difficulty.unique()


## stemmer

In [ ]:
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Initialize French stemmer
stemmer = SnowballStemmer("french")

# French stopwords from NLTK
french_stopwords = set(stopwords.words('french'))

def clean_text_french(text):
    # Clean and prepare text
    text = re.sub("[^\w\s]", " ", text)
    # Split text into words, filter stopwords, apply stemming, and convert to lower case
    return " ".join([stemmer.stem(word) for word in text.split() if word.lower() not in french_stopwords]).lower()

'''
# Sample data
data = pd.DataFrame({
    'text': ["Café au centre de Paris", "Il fait un beau jour!", "résumé et portfolio"]
})
'''
df_tr_03 = df_tr_02.copy()

# Applying the French text cleaning function
df_tr_03['cleaned'] = df_tr_03['sentence'].apply(clean_text_french)
print(df_tr_03.cleaned)

## lemmatize

In [ ]:
import spacy

# Load French Spacy model
nlp = spacy.load("fr_core_news_sm")

def clean_text_french_lemmatized(text):
    # Process text using Spacy
    doc = nlp(text)
    # Filter stopwords and punctuation, then lemmatize
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct]).lower()

df_tr_04 = df_tr_02.copy()

# Applying the French text cleaning function with lemmatization
df_tr_04['cleaned'] = df_tr_04['sentence'].apply(clean_text_french_lemmatized)
print(df_tr_04.cleaned)


 ## ML

### Camembert embeddings

In [ ]:
#camembert embeddings

import pandas as pd
import torch
from transformers import CamembertTokenizer, CamembertModel
from sklearn.preprocessing import StandardScaler

df_tr_20 = df_tr_02.copy()

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

def get_embeddings(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    encoded_input = {key: val.to(model.device) for key, val in encoded_input.items()}
    with torch.no_grad():
        outputs = model(**encoded_input)
    #hidden state
    embeddings = outputs.last_hidden_state
    embeddings = embeddings.mean(dim=1)
    return embeddings.squeeze().cpu().tolist()

df_tr_20['embeddings'] = df_tr_20['sentence'].apply(get_embeddings)
#print(df_tr_20['embeddings'].shape())
df_tr_20.head()


In [ ]:
print(df_tr_20['embeddings'].shape)

### Multi input NN

In [ ]:
import json
#si weight and biases account
'''
import wandb
run = wandb.init(
    project='Cam02', 
    name="001", 
)
'''

df_tr_08 = df_tr_01.copy()
label_encoder = LabelEncoder()
df_tr_08['difficulty'] = label_encoder.fit_transform(df_tr_08['difficulty'])

nlp = spacy.load('fr_core_news_sm')

def extract_features(doc):
    # Lexical features
    words = [token.text for token in doc if token.is_alpha]
    avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
    type_token_ratio = len(set(words)) / len(words) if words else 0
    
    # Lexical density
    content_pos_tags = {'NOUN', 'VERB', 'ADJ', 'ADV'}
    content_words = [token for token in doc if token.pos_ in content_pos_tags]
    lexical_density = len(content_words) / len(words) if words else 0
    
    # Syntactic features
    sentences = list(doc.sents)
    avg_sentence_length = sum(len(sentence) for sentence in sentences) / len(sentences) if sentences else 0
    
    #Syntactic complexity: estimated by counting finite verbs in each sentence
    clauses_per_sentence = [sum(1 for token in sentence if token.tag_ in ['VERB', 'AUX']) for sentence in sentences]
    avg_clauses_per_sentence = sum(clauses_per_sentence) / len(clauses_per_sentence) if clauses_per_sentence else 0
    
    noun_phrases = list(doc.noun_chunks)
    avg_noun_phrase_length = sum(len(np) for np in noun_phrases) / len(noun_phrases) if noun_phrases else 0
    
    #Count of nouns, verbs, adjectives
    num_nouns = sum(1 for token in doc if token.pos_ == 'NOUN')
    num_verbs = sum(1 for token in doc if token.pos_ == 'VERB')
    num_adjectives = sum(1 for token in doc if token.pos_ == 'ADJ')
    
    return {
        'avg_word_length': avg_word_length,
        'type_token_ratio': type_token_ratio,
        'lexical_density': lexical_density,
        'avg_sentence_length': avg_sentence_length,
        'avg_clauses_per_sentence': avg_clauses_per_sentence,
        'avg_noun_phrase_length': avg_noun_phrase_length,
        'num_nouns': num_nouns,
        'num_verbs': num_verbs,
        'num_adjectives': num_adjectives
    }

def add_features_to_dataframe(df, text_column):
    df['spacy_doc'] = df[text_column].apply(nlp)  # Convert text to spacy Doc objects
    features_df = df['spacy_doc'].apply(extract_features).apply(pd.Series)
    new_df = pd.concat([df, features_df], axis=1)
    new_df.drop('spacy_doc', axis=1, inplace=True)
    return new_df

df_with_features_01 = add_features_to_dataframe(df_tr_08, 'sentence')

df_with_features_02 = df_with_features_01.copy()
df_with_features_02['embeddings'] = df_tr_20['embeddings'] #df_tr_20 les embed camembert
df_with_features_02 = df_with_features_02.drop('sentence', axis=1)
df_with_features_02 = df_with_features_02.drop('id', axis=1)
df_only_features_02 = df_with_features_02.drop('difficulty', axis=1)

df_y_01 = df_tr_05_02.copy()
df_y_01['difficulty'] = df_y_01['difficulty'].astype(float)
print('----------------------------------------------')

X_train, X_test, y_train, y_test = train_test_split(df_only_features_02, df_y_01['difficulty'], test_size=0.1, random_state=42)

X_train_embeddings = X_train['embeddings'].tolist()
X_train_features = X_train.drop('embeddings', axis=1)

X_test_embeddings = X_test['embeddings'].tolist()
X_test_features = X_test.drop('embeddings', axis=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.fit_transform(X_test_features) ######################## transform

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_features.columns, index=X_train_features.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_features.columns, index=X_test_features.index)
#---------------------------------------------------------
#----------------------------------------------------------------
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train_scaled, y_train)

importances = forest.feature_importances_
important_feature_indices = np.argsort(importances)[::-1][:4]  # Select top 3 features for simplicity

important_features = X_train_scaled.columns[important_feature_indices]

X_train_numerical_selected = X_train_scaled[important_features]
X_test_numerical_selected = X_test_scaled[important_features]

'''
pca = PCA(n_components=800)  # Reduce to 50 components
X_train_embeddings_reduced = pca.fit_transform(X_train_embeddings)
X_test_embeddings_reduced = pca.transform(X_test_embeddings)
'''
#----------------------------------------------------------------
'''
numeric_input = Input(shape=(X_train_numerical_selected.shape[1],), name='numeric_data')
embedding_input = Input(shape=(len(X_train_embeddings[0]),), name='embedding_data')
'''

print("Shape of X_train_numerical_selected:", X_train_numerical_selected.shape)

# Input layers
input_numerical = Input(shape=(X_train_numerical_selected.shape[1],), name='numeric_input')
#input_embeddings = Input(shape=(800,), name='embedding_input')
input_embeddings = Input(shape=(np.array(X_train_embeddings).shape[1],), name='embedding_input')

# Processing paths
path_numerical = Dense(50, activation='relu')(input_numerical)
path_embeddings = Dense(50, activation='relu')(input_embeddings)

# Concatenate data paths
concatenated = Concatenate()([path_numerical, path_embeddings])

# Additional Dense layers
dense = Dense(50, activation='relu')(concatenated)
dense = Dense(25, activation='relu')(dense) 
#dropout = Dropout(0.5)(dense)
output = Dense(6, activation='softmax')(dense)

optimizer = AdamW(learning_rate=0.0001) # 0.01

model = Model(inputs=[input_numerical, input_embeddings], outputs=output)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

X_train_embeddings_reduced_np = np.array(X_train_embeddings)
X_test_embeddings_reduced_np = np.array(X_test_embeddings)

#print("Shape of X_train_numerical_selected:", X_train_numerical_selected.shape)
#print("Shape of X_train_embeddings_reduced:", X_train_embeddings_reduced.shape)

model.fit([X_train_numerical_selected, X_train_embeddings_reduced_np], y_train, epochs=8, validation_split=0.1, batch_size=1) #50 0.1 4

In [ ]:
test_loss, test_accuracy = model.evaluate([(X_test_numerical_selected, X_test_embeddings_reduced_np)], y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
model.save('my_model.h5')

### prepare unabelled data

In [ ]:
#save unte

df_tr_un_08 = df_unte_01.copy()

label_encoder = LabelEncoder()

nlp = spacy.load('fr_core_news_sm')

#WIP use only the selected features
'''
def extract_features(doc):
    # Lexical features
    words = [token.text for token in doc if token.is_alpha]
    avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
    type_token_ratio = len(set(words)) / len(words) if words else 0
    
    # Lexical density: proportion of content words (nouns, verbs, adjectives, adverbs)
    content_pos_tags = {'NOUN', 'VERB', 'ADJ', 'ADV'}
    content_words = [token for token in doc if token.pos_ in content_pos_tags]
    lexical_density = len(content_words) / len(words) if words else 0
    
    # Syntactic features
    sentences = list(doc.sents)
    avg_sentence_length = sum(len(sentence) for sentence in sentences) / len(sentences) if sentences else 0
    
    # Syntactic complexity: estimated by counting finite verbs in each sentence
    clauses_per_sentence = [sum(1 for token in sentence if token.tag_ in ['VERB', 'AUX']) for sentence in sentences]
    avg_clauses_per_sentence = sum(clauses_per_sentence) / len(clauses_per_sentence) if clauses_per_sentence else 0
    
    noun_phrases = list(doc.noun_chunks)
    avg_noun_phrase_length = sum(len(np) for np in noun_phrases) / len(noun_phrases) if noun_phrases else 0
    
    # Semantic features: Count of nouns, verbs, adjectives
    num_nouns = sum(1 for token in doc if token.pos_ == 'NOUN')
    num_verbs = sum(1 for token in doc if token.pos_ == 'VERB')
    num_adjectives = sum(1 for token in doc if token.pos_ == 'ADJ')
    
    # Return all features as a dictionary
    return {
        'avg_word_length': avg_word_length,
        'type_token_ratio': type_token_ratio,
        'lexical_density': lexical_density,
        'avg_sentence_length': avg_sentence_length,
        'avg_clauses_per_sentence': avg_clauses_per_sentence,
        'avg_noun_phrase_length': avg_noun_phrase_length,
        'num_nouns': num_nouns,
        'num_verbs': num_verbs,
        'num_adjectives': num_adjectives
    }
'''
def add_features_to_dataframe(df, text_column):
    df['spacy_doc'] = df[text_column].apply(nlp)
    features_df = df['spacy_doc'].apply(extract_features).apply(pd.Series)
    new_df = pd.concat([df, features_df], axis=1)
    new_df.drop('spacy_doc', axis=1, inplace=True)
    return new_df

df_un_with_features_01 = add_features_to_dataframe(df_tr_un_08, 'sentence')
df_un_with_features_02 = df_un_with_features_01.copy()
df_un_with_features_02['embeddings'] = df_tr_20['embeddings']
df_un_with_features_02 = df_un_with_features_02.drop('sentence', axis=1)
df_un_with_features_02 = df_un_with_features_02.drop('id', axis=1)

X_train_embeddings = df_un_with_features_02['embeddings'].tolist()
X_train_features = df_un_with_features_02.drop('embeddings', axis=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_features.columns, index=X_train_features.index)
'''
# Get feature importance and select the most important ones
importances = forest.feature_importances_
important_feature_indices = np.argsort(importances)[::-1][:3]  # Select top N features

important_features = X_train_scaled.columns[important_feature_indices]

# Filter your train and test sets
X_train_numerical_selected = X_train_scaled[:, important_feature_indices]
X_test_numerical_selected = X_test_scaled[:, important_feature_indices]
'''
# Get feature importance and sort them
#importances = forest.feature_importances_
#important_feature_indices = np.argsort(importances)[::-1][:4]  # Select top 3 features for simplicity

important_features = X_train_scaled.columns[important_feature_indices]

X_train_numerical_selected = X_train_scaled[important_features]

In [ ]:
loaded_model_h5 = tf.keras.models.load_model('my_model.h5')
predictions = loaded_model_h5.predict([X_new_numerical, X_new_embeddings])

In [ ]:
# Predict using the trainer
predicted_labels = np.argmax(predictions.predictions, axis=1)

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_labels]

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predicted_difficulties
})

submission_df.to_csv('F:\projects\Matthias\proj_01\cam01\submission_CamemBERT_018.csv', index=False)

In [ ]:
print("Selected features:", important_features)

In [ ]:
print(X_train_numerical_selected.shape, np.array(X_train_embeddings).shape, y_train.shape)

In [ ]:
print(X_test_numerical_selected.shape, np.array(X_test_embeddings).shape, y_test.shape)

### Hypermodel

In [ ]:

'''
def apply_pca(X, n_components=666): #50
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca
'''

class FullHyperModel(HyperModel):
    def __init__(self, X_train_num, X_train_emb, y_train):
        self.X_train_num = X_train_num
        self.X_train_emb = X_train_emb
        self.y_train = y_train

    def build(self, hp):
        # PCA for embeddings
        #n_components = hp.Int('n_components', min_value=100, max_value=1000, step=20)
        
        '''
        pca = PCA(n_components=n_components)
        X_train_emb_pca = pca.fit_transform(self.X_train_emb)
        '''

    
        input_emb = Input(shape=(self.X_train_emb.shape[1],), name='input_emb')
        emb_units = hp.Int('emb_units', min_value=20, max_value=256, step=32)
        emb_path = Dense(emb_units, activation='relu')(input_emb)

        # Input for numerical features
        input_num = Input(shape=(self.X_train_num.shape[1],), name='input_num')
        num_units = hp.Int('num_units', min_value=20, max_value=256, step=32)
        num_path = Dense(num_units, activation='relu')(input_num)

        # Concatenate and further processing
        concatenated = Concatenate()([num_path, emb_path])
        final_units = hp.Int('final_units', min_value=10, max_value=256, step=32)
        dense = Dense(final_units, activation='relu')(concatenated)
        
        output = Dense(6, activation='softmax')(dense)  
        
        # Hyperparameters for the optimizer
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        weight_decay = hp.Float('weight_decay', min_value=1e-5, max_value=1e-2, step=1e-5)

        # Model compilation
        model = Model(inputs=[input_num, input_emb], outputs=output)
        '''
        model.compile(
            optimizer=AdamW(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]),weight_decay=hp.Float('weight_decay', min_value=0.01, max_value=0.1, step=0.01)),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        '''
        model.compile(
            optimizer=tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
            # optimizer=AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        model.summary()
        return model
    

    def fit(self, hp, model, *args, **kwargs):
        batch_size = hp.Int('batch_size', min_value=1, max_value=16)
        #n_components = hp.get('n_components')
        #pca = PCA(n_components=n_components)
        #X_train_emb_pca = pca.fit_transform(self.X_train_emb)
        print("Training data shape:", self.X_train_num.shape)
        print("Training embeddings shape:", self.X_train_emb.shape)
        print("Labels shape:", self.y_train.shape)
        print("batch size:", batch_size)
        return model.fit([self.X_train_num, self.X_train_emb], self.y_train, batch_size=batch_size, *args, **kwargs)
    
tuner = RandomSearch(
    FullHyperModel(X_train_numerical_selected, np.array(X_train_embeddings), y_train),
    objective='val_accuracy',
    max_trials=100, 
    executions_per_trial=4, 
    directory='my_tuning_dir',
    project_name='full_model_tuning_11'
)

tuner.search(epochs=30, validation_split=0.1)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('best_model_03.h5')

In [ ]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

batch_size = best_hp.get('batch_size')
learning_rate = best_hp.get('learning_rate')
weight_decay = best_hp.get('weight_decay')


# Optionally, directly fetch the best model using the best hyperparameters
#best_model = tuner.get_best_models(num_models=1)[0]

hyperparameters = {
    'batch_size': best_hp.get('batch_size'),
    'learning_rate': best_hp.get('learning_rate'),
    'weight_decay': best_hp.get('weight_decay'),
    #'n_components': best_hp.get('n_components')
}

with open('best_hyperparameters.json', 'w') as f:
    json.dump(hyperparameters, f)

In [ ]:
print(X_train_numerical_selected.shape)
print(len(X_train_embeddings))

In [ ]:
model = load_model('best_model_03.h5')
# Load the hyperparameters
with open('best_hyperparameters.json', 'r') as f:
    hyperparams = json.load(f)

print(hyperparams['batch_size'], hyperparams['learning_rate'], hyperparams['weight_decay'])

#pca = PCA(n_components=700)  # Reduce to 50 components
#X_test_embeddings_reduced = pca.fit_transform(X_test_embeddings)

In [ ]:
#model.predict(X_test_numerical_selected, X_test_embeddings, y_test)
test_loss, test_accuracy = model.evaluate([(X_test_numerical_selected, pd.DataFrame(X_test_embeddings))], y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

predictions = best_model.predict([X_test_numerical_selected, pd.DataFrame(X_test_embeddings)])
predicted_classes = np.argmax(predictions, axis=1)
difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_classes]
original_difficulties = [difficulty_levels[label] for label in y_test]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()